In [1]:
from postgisUtil import *
from matplotlib import pyplot as plt
import pickle
import os

In [ ]:
def db_init():
    directory_name = "ASO_Lidar/"
    basin_names  = ["Merced", "Tuolumne"]
    feature_names = ["DEM", "SLP", "ASP", "NOR", "CHM", "VEG"]
    kNN_db = postgisUtil("knn_project", "zeshi")
#     kNN_db.create_schema("topo")
    for basin in basin_names:
        print basin
        for i, feature in enumerate(feature_names):
            print feature
            file_name = directory_name + basin + "_500m_" + feature + ".tif"
            if i == 0:
                kNN_db.load_map_to_db(file_name, feature, "topo", basin.lower(), table_exist=False)
            else:
                kNN_db.load_map_to_db(file_name, feature, "topo", basin.lower())
    kNN_db.close()

In [ ]:
def db_geom_init():
    knn_db = postgisUtil("knn_project", "zeshi")
    knn_db.create_schema('sensors')
    knn_db.create_table('site_locs', 'sensors', ['site_id'], ['SERIAL'])
    knn_db.add_geometry_column('sensors', 'site_locs', 'site_coords', 32611, 'POINT', 2, use_typemod='false')
    location = pickle.load(open('kNN_training_testing/sensor_idx.p'))
    test_coords = knn_db.convert_idx_to_coords('DEM', location[1], location[0], 'topo', 'merced')
    print test_coords
    knn_db.add_geoms_to_table(test_coords[0], test_coords[1], 32611, 'sensors', 'site_locs', 'site_coords')
    map_pixel_values = knn_db.geoms_table_to_map_pixel_values(['DEM', 'ASP'], 'sensors', 'site_locs', 'site_coords', 'topo', 'merced')
    print map_pixel_values
    knn_db.close()

In [2]:
def db_swe_init(site_name, schema_exist=True):
    directory_name = "ASO_Lidar/"
    knn_db = postgisUtil("knn_project", "zeshi")
    if not schema_exist:
        knn_db.create_schema("swe_lidar")
    lidar_fns = os.listdir("ASO_Lidar/")
    if site_name.lower()=="merced":
        site_name_ab = "MB"
    elif site_name.lower()=="tuolumne":
        site_name_ab = "TB"
    lidar_site_fns = [temp_str for temp_str in lidar_fns if temp_str.startswith(site_name_ab)]
    lidar_site_date = [temp_str[2:10] for temp_str in lidar_site_fns]
    for i, lidar_site_fn in enumerate(lidar_site_fns):
        file_name = directory_name + site_name_ab + lidar_site_date[i] + "_500m.tif"
        print file_name
        if i == 0:
            knn_db.load_map_to_db(file_name, lidar_site_date[i], "swe_lidar", site_name, table_exist=False)
        else:
            knn_db.load_map_to_db(file_name, lidar_site_date[i], "swe_lidar", site_name)
    knn_db.close()

In [ ]:
db_swe_init("merced", schema_exist=False)

In [3]:
db_swe_init("tuolumne", schema_exist=True)

ASO_Lidar/TB20140428_500m.tif
ASO_Lidar/TB20140323_500m.tif
ASO_Lidar/TB20140511_500m.tif
ASO_Lidar/TB20140502_500m.tif
ASO_Lidar/TB20140413_500m.tif
ASO_Lidar/TB20140407_500m.tif
ASO_Lidar/TB20140420_500m.tif


In [ ]:
class Solution(object):
    def longestCommonPrefix(self, strs):
        """
        :type strs: List[str]
        :rtype: str
        """
        suffix_strings_array = self.suffix_tree_construct([], strs)
        print suffix_strings_array
        prefix_array = []
        for i in range(len(suffix_strings_array)-1):
            j = 0
            while j < len(suffix_strings_array[i]) and j < len(suffix_strings_array[i + 1]):
                if suffix_strings_array[i][j] == suffix_strings_array[i+1][j]:
                    j += 1
                else:
                    break
            prefix_array.append(j)
        lcp_length = max(prefix_array)
        lcp_length_idx = prefix_array.index(lcp_length)
        lcp_string = suffix_strings_array[lcp_length_idx][0:lcp_length]
        return lcp_string
        
    def suffix_tree_construct(self, current_strings, remaining_strings):
        if current_strings==[]:
            current_strings.append(remaining_strings.pop(0))
        else:
            temp_string = remaining_strings.pop(0)
            for i in range(len(current_strings)-1, -1, -1):
                compare_string = current_strings[i]
                insert = True
                j = 0
                while j < len(compare_string) and j < len(temp_string):
                    if temp_string[j] < compare_string[j]:
                        insert = False
                        break
                    j += 1
                if j==len(temp_string) and j < len(compare_string):
                    insert = False
                if insert:
                    current_strings.insert(i+1, temp_string)
                    break
                if not insert and i == 0:
                    current_strings.insert(i, temp_string)
        if remaining_strings == []:
            return current_strings
        else:
            current_strings = self.suffix_tree_construct(current_strings, remaining_strings)
        return current_strings

In [ ]:
obj = Solution()
print obj.longestCommonPrefix(["abc", "ab", "abba"])

In [ ]:
print os.listdir("ASO_Lidar/")

In [45]:
def comb(digit_list, k):
    temp_list = []
    if len(digit_list) == k:
        return [digit_list]
    if k == 0:
        return []
    else:
        temp_result_1 = comb(digit_list[1:], k-1)
        temp_result_0 = comb(digit_list[1:], k)
        if temp_result_1:
            for temp in temp_result_1:
                temp_list.append([digit_list[0]]+temp)
        else:
            temp_list.append([digit_list[0]])

        for temp in temp_result_0:
            temp_list.append(temp)
    return temp_list

In [46]:
print comb(range(12), 1)

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11]]


In [49]:
from itertools import combinations
combin = combinations(range(12), 0)
for a in combin:
    print a

()


In [ ]:
class ListNode(object):
    def __init__(self, x):
        self.val = x
        self.next = None

class Solution(object):
    def mergeTwoLists(self, l1, l2):
        """
        :type l1: ListNode
        :type l2: ListNode
        :rtype: ListNode
        """
        pointer1 = l1
        pointer2 = l2
        
        if pointer2.val <= l1.val:
            tempNode = pointer2
            tempNode.next = pointer1
            pointer1 = tempNode
            l1 = pointer1
            pointer2 = pointer2.next
 
        while pointer2 is not None and pointer1.next is not None:
            print point1.val
            if pointer2.val >= pointer1.val and pointer2.val <= pointer1.next.val:
                tempNode = pointer2
                tempNode.next = pointer1.next
                pointer1.next = tempNode
                pointer2 = pointer2.next
                pointer1 = pointer1.next
            else:
                pointer1 = pointer1.next
        if pointer2:
            pointer1.next = pointer2
        return l1

In [ ]:
l1 = ListNode(1)
second = ListNode(2)
second.next = listNode(4)
l1.next = second
l2 = ListN
sol = Solution()
mergeTwoLists